In [2]:
# !pip install transformers==4.48.0 datasets sentencepiece

In [3]:
import transformers

print(transformers.__version__)

4.48.0


In [4]:
from transformers import pipeline

In [5]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [6]:
classifier(
    [
        "오늘 날씨가 너무 좋았어 점심먹고 산책하니 너무 기분이 좋더라",
        "I've fought with my friend she shouted at me",
        "맥북 컴퓨터 M5 발표"
    ]
)

[{'label': 'POSITIVE', 'score': 0.7537334561347961},
 {'label': 'NEGATIVE', 'score': 0.9564605951309204},
 {'label': 'NEGATIVE', 'score': 0.5040510892868042}]

In [7]:
from transformers import AutoTokenizer

In [8]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [9]:
raw_inputs = [
    "오늘 날씨가 너무 좋았어 점심먹고 산책하니 너무 기분이 좋더라",
    "I've fought with my friend she shouted at me"
]

In [10]:
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

In [11]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [12]:
type(inputs["input_ids"])

torch.Tensor

In [13]:
inputs["input_ids"]

tensor([[  101,  1463, 30011, 29992, 30017, 30022,  1456, 30006, 30022, 29998,
         30019, 29991, 30006,  1456, 30008, 29995, 30014,   100,  1464, 30008,
         30023, 29997, 30019, 30023, 29995, 30008, 30020, 29991, 30011,  1461,
         30006, 30021, 30001, 30007, 30020, 30005, 30006, 29992, 30019,  1456,
         30008, 29995, 30014,  1455, 30019, 29996, 30014, 30021, 29999, 30019,
           100,   102],
        [  101,  1045,  1005,  2310,  4061,  2007,  2026,  2767,  2016,  6626,
          2012,  2033,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])

In [14]:
inputs["attention_mask"]

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

In [15]:
from transformers import AutoModelForSequenceClassification

In [16]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [17]:
# 토크나이즈 결과를 모델에 입력으로 넣어서 logit값 계산
outputs = model(**inputs)

In [18]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.4739,  0.6447],
        [ 1.6939, -1.3957]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [19]:
outputs.logits

tensor([[-0.4739,  0.6447],
        [ 1.6939, -1.3957]], grad_fn=<AddmmBackward0>)

In [20]:
import torch

In [21]:
# logit에 softmax 함수를 적용하여 확률값으로 변환
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [22]:
predictions

tensor([[0.2463, 0.7537],
        [0.9565, 0.0435]], grad_fn=<SoftmaxBackward0>)

In [23]:
0.2463 + 0.7537, 0.9565 + 0.0435

(1.0, 1.0)

In [24]:
# argmax를 사용하여 각 문장별 확률이 제일 큰 label의 idx 계산
predictions.argmax(dim=-1)

tensor([1, 0])

In [25]:
predictions.argmax(dim=-1).tolist()

[1, 0]

In [27]:
# idx를 label로 변환하기 위한 모델의 config 확인
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [28]:
# 가장 높은 확률값의 idx별 label값 변환
[model.config.id2label[item] for item in predictions.argmax(dim=-1).tolist()]

['POSITIVE', 'NEGATIVE']

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = "kakaocorp/kanana-nano-2.1b-instruct"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
).to("cuda")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [ ]:
prompt = "오늘은 아주 화창하고 시원한 가을 날씨였어 오늘 날씨에 맞는 야식 추천해줘"

In [ ]:
messages = [
    {
        "role":"system",
        "content":"You are a helpful AI assistant developed by Kakao."
    },
    {
        "role":"user",
        "content":prompt
    }
]

In [ ]:
chat_template_example = tokenizer.apply_chat_template(
    messages,
    tokenize=False
)
chat_template_example

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant developed by Kakao.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n오늘은 아주 화창하고 시원한 가을 날씨였어 오늘 날씨에 맞는 야식 추천해줘<|eot_id|>'

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

In [ ]:
input_ids

tensor([[128000, 128006,   9125, 128007,    271, 109659, 104449,  18359, 107744,
          57002, 109584,    167, 107569, 104508,  17835, 107973,     11, 111964,
          21028, 110154, 102772,  23955, 101555, 102199, 102766,  18359, 118667,
          59269,    246, 128009, 128006,    882, 128007,    271,  58368, 105622,
          34804, 117454, 104323, 106646, 101360,  45618,  55421,  24486,  36609,
          18359, 105605, 107497, 101574,  32179, 111128, 105605, 107497,  19954,
         107625,  16969, 108332,  77437, 109336,  34983,  59269,    246, 128009,
         128006,  78191, 128007,    271]], device='cuda:0')

In [ ]:
output = model.generate(
    input_ids=input_ids,
    max_new_tokens=200,
    do_sample=False
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
# output

In [ ]:
# do_sample=True
print(tokenizer.decode(output[0], skip_special_tokens=True))

system

You are a helpful AI assistant developed by Kakao.user

오늘은 아주 화창하고 시원한 가을 날씨였어 오늘 날씨에 맞는 야식 추천해줘assistant

가을 날씨에 딱 맞는 시원하고 따뜻한 야식 추천해드릴게요!

### 가을 야식 추천 리스트

1. **호박전**
   - **재료:** 호박, 밀가루, 계란, 참기름
   - **조리법:** 호박을 채 썰어 밀가루와 계란을 입혀 팬에 부쳐내면 고소하고 풍부한 맛이 납니다.

2. **김치전**
   - **재료:** 김치, 밀가루, 물, 참기름
   - **조리법:** 잘 익은 김치와 밀가루를 섞어 반죽한 후, 팬에 부쳐내면 맛있는 김치전이 완성됩니다.

3. **치킨&맥주**
   - 가을 밤에 빠질 수 없는 치킨과 맥주 세트!
   - 각종 치킨


In [29]:
# do_sample=False


-  아래 두 가지 생성 결과는 다른 사람들이 생성한 결과이며, `do_sample=False`로 할 시 greedy로 생성되기 때문에 생성 결과가 같음
```
1. **치맥 (치킨과 맥주)**
   - 가을 밤, 치맥은 빠질 수 없는 조합이죠. 바삭한 치킨과 시원한 맥주가 완벽한 조화를 이룹니다.

2. **전복죽**
   - 가을철에는 소화가 잘 되는 음식이 좋아요. 전복죽은 부드럽고 고소해서 속을 편안하게 해줍니다.

3. **오징어볶음**
   - 매콤하게 조리된 오징어볶음은 가을의 매운 맛을 즐기기에 좋습니다. 고추장과 고춧가루가 어우러져 입맛을 돋워줍니다.
```

```
1. **치맥 (치킨과 맥주)**
   - 가을 밤, 치맥은 빠질 수 없는 조합이죠. 바삭한 치킨과 시원한 맥주가 완벽한 조화를 이룹니다.

2. **전복죽**
   - 가을철에는 소화가 잘 되는 음식이 좋아요. 전복죽은 부드럽고 고소해서 속을 편안하게 해줍니다.

3. **오징어볶음**
   - 매콤하게 조리된 오징어볶음은 가을의 매운 맛을 즐기기에 좋습니다. 고추장과 고춧가루가 어우러져 입맛을 돋워줍니다.
   
```
